In [26]:
# -*- coding: utf-8 -*-
# 
# Purpose: 自中華民國氣象局 Open Data 取得自動雨量站 每10分鐘雨量資料 
# Remark: 解析json資料有參考 https://github.com/leafwind/cwb-cache 非常感謝
#
import logging
import pandas as pd
import requests
from datetime import datetime
import sqlalchemy

#氣象局API網址
CWB_URL = 'http://opendata.cwb.gov.tw/api/v1/rest/datastore/'
#自動雨量站 資料集代號
DATA_ID = 'O-A0002-001'
#氣象局 OPEN DATA 授權碼，需要自行修改
AUTH_KEY = 'CWB-OOOOOOOOO'
#目的地 SQL Server 的連結字串，需要自行修改
SQL_CONNECTION_STRING = "mssql+pyodbc://[帳號]:[密碼]@[SQL Sever]:1433/[Database]?driver=SQL+Server+Native+Client+11.0"
#目的地 SQL Server Table名稱，需要自行修改
DEST_TABLE = 'tb_Rainfall_API'

In [27]:
#取得API 回傳的 JSON資料
def get_data_from_cwb(data_id, auth_key, params={}):
    '''limit, offset, format, locationName, elementName, sort'''
    logging.info('getting data from CWB...')

    dest_url = CWB_URL + '{}'.format(data_id)
    r = requests.get(dest_url, headers={'Authorization': auth_key})
    params_list = ['{}={}'.format(key, params[key]) for key in params]
    params_str = '?' + '&'.join(params_list)
    dest_url += params_str
    logging.debug('dest_url: {}'.format(dest_url))
    
    if r.status_code != 200:
        logging.error('r.status_code: {}'.format(r.status_code))
        return None

    data = r.json()
    
    if data.get('success') != 'true':
        return None
    return data

#根據json資料解析成 dataframe
def parse_json_to_dataframe(data):
    #logging.info('parsing {} ...'.format(data['records']['datasetDescription'].encode('utf-8')))
    
    #準備結果的 dataframe
    columns = ['stationId','locationName','lat','lon', 'obstime','ELEV','RAIN','MIN_10','HOUR_3','HOUR_6','HOUR_12','HOUR_24','NOW']
    df = pd.DataFrame(columns=columns)
    dataDic = {}
    locations = data['records']['location']
    row = -1
    for l in locations:
        row = row + 1
        dataDic['stationId'] = l['stationId']
        dataDic['locationName'] = l['locationName']
        dataDic['obstime'] = l['time']['obsTime']
        dataDic['lat'] = l['lat']
        dataDic['lon'] = l['lon']
        factors = l['weatherElement']
        for f in factors:
            factor_name = f['elementName']
            dataDic[factor_name] = f['elementValue']
        for key in dataDic.keys():
            df.loc[row,key] = dataDic[key]
        

    return df 


In [30]:
if __name__ == '__main__':
    json_data = get_data_from_cwb(DATA_ID, AUTH_KEY, {})
    df = parse_json_to_dataframe(json_data)
    df['InsertDatetime']=datetime.now()

In [31]:
    #確認結果
    df[0:6]

,stationId,locationName,lat,lon,obstime,ELEV,RAIN,MIN_10,HOUR_3,HOUR_6,HOUR_12,HOUR_24,NOW,InsertDatetime
0,C0A560,福山,24.7783,121.4946,2018-05-27 12:10:00,405.00,-998.00,-998.00,-998.00,-998.00,0.00,0.00,0.00,2018-05-27 12:22:12.648240
1,C0X190,安平,22.9950,120.1441,2018-05-27 12:10:00,10.00,-998.00,-998.00,-998.00,-998.00,0.00,0.00,0.00,2018-05-27 12:22:12.648240
2,C1F9E1,龍安,24.1619,120.8239,2018-05-27 12:10:00,563.00,-998.00,-998.00,-998.00,-998.00,0.00,0.00,0.00,2018-05-27 12:22:12.648240
3,467480,嘉義,23.4977,120.4248,2018-05-27 12:10:00,26.90,-998.00,-998.00,-998.00,-998.00,0.00,0.00,0.00,2018-05-27 12:22:12.648240
4,C0M730,嘉義市東區,23.4594,120.4524,2018-05-27 12:10:00,40.00,-998.00,-998.00,-998.00,-998.00,0.00,0.00,0.00,2018-05-27 12:22:12.648240
5,C0A650,火燒寮,25.0044,121.7346,2018-05-27 12:10:00,287.00,-998.00,-998.00,-998.00,-998.00,0.00,0.00,0.00,2018-05-27 12:22:12.648240


In [32]:
    #寫入資料庫
    engine = sqlalchemy.create_engine(SQL_CONNECTION_STRING)
    conn = engine.connect()
    
    df.to_sql(DEST_TABLE,engine,if_exists='append')
    #確認資料庫結果
    rs = conn.execute('SELECT TOP 5 * FROM ' + DEST_TABLE +' with (nolock) ORDER BY InsertDatetime desc ')
    _result = pd.DataFrame(rs.fetchall())
    _result.columns = rs.keys()
    conn.close()
    
    _result

,index,stationId,locationName,lat,lon,obstime,ELEV,RAIN,MIN_10,HOUR_3,HOUR_6,HOUR_12,HOUR_24,NOW,InsertDatetime
0,0,C0A560,福山,24.7783,121.4946,2018-05-27 12:10:00,405.00,-998.00,-998.00,-998.00,-998.00,0.00,0.00,0.00,2018-05-27 12:22:12
1,1,C0X190,安平,22.9950,120.1441,2018-05-27 12:10:00,10.00,-998.00,-998.00,-998.00,-998.00,0.00,0.00,0.00,2018-05-27 12:22:12
2,2,C1F9E1,龍安,24.1619,120.8239,2018-05-27 12:10:00,563.00,-998.00,-998.00,-998.00,-998.00,0.00,0.00,0.00,2018-05-27 12:22:12
3,3,467480,嘉義,23.4977,120.4248,2018-05-27 12:10:00,26.90,-998.00,-998.00,-998.00,-998.00,0.00,0.00,0.00,2018-05-27 12:22:12
4,4,C0M730,嘉義市東區,23.4594,120.4524,2018-05-27 12:10:00,40.00,-998.00,-998.00,-998.00,-998.00,0.00,0.00,0.00,2018-05-27 12:22:12
